In [ ]:
import os

import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

from diffusers.training_utils import set_seed
from src.prepare_easy import prepare_prompts_wiki398

torch.backends.cuda.matmul.allow_tf32 = True
torch._inductor.config.conv_1x1_as_mm = True
torch._inductor.config.coordinate_descent_tuning = True
torch._inductor.config.epilogue_fusion = False
torch._inductor.config.coordinate_descent_check_all_directions = True

import random

import clip
import pandas as pd
from matplotlib import pyplot as plt
from pytorch_msssim import ssim

from diffusers import IFPipeline, IFSuperResolutionPipeline
from src.eval.basic_metrics import calculate_mse, calculate_psnr_from_mse
from src.eval.clipscore import clip_metrics, extract_all_images
from src.eval.ocr_eval import get_ocr_easyocr, get_text_easyocr, ocr_metrics
from src.eval.text_distance import get_levenshtein_distances

plt.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "font.size": 15,  # Set font size to 11pt
        "axes.labelsize": 15,  # -> axis labels
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "legend.fontsize": 12,
        "lines.linewidth": 2,
        "text.usetex": False,
        "pgf.rcfonts": False,
    }
)

In [ ]:
SDXL_MODEL_NAME_OR_PATH = "DeepFloyd/IF-I-XL-v1.0"
SR_MODEL_NAME_OR_PATH = "DeepFloyd/IF-II-L-v1.0"
SEED = 42
N_SAMPLES_PER_PROMPT = 1
BATCH_SIZE = 8
DEVICE = "cuda"
NUM_INFERENCE_STEPS = 50
ATTENTIONS_TO_PATCH = list(range(21))

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].shape[1], imgs[0].shape[0]
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        img = img.astype(np.uint8)
        grid.paste(Image.fromarray(img), box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
pipe = IFPipeline.from_pretrained(
    SDXL_MODEL_NAME_OR_PATH,
    variant="fp16",
    use_safetensors=True,
    token=os.environ.get("HF_TOKEN"),
    torch_dtype=torch.float32 if DEVICE == "cpu" else torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
    watermarker=None,
).to(DEVICE)

pipe.set_progress_bar_config(disable=True)

pipe_upsample = IFSuperResolutionPipeline.from_pretrained(
    SR_MODEL_NAME_OR_PATH,
    variant="fp16",
    use_safetensors=True,
    token=os.environ.get("HF_TOKEN"),
    torch_dtype=torch.float32 if DEVICE == "cpu" else torch.float16,
)
pipe_upsample.set_progress_bar_config(disable=True)

In [ ]:
generator = torch.Generator().manual_seed(SEED)

In [ ]:
noises = torch.randn(
    (N_SAMPLES_PER_PROMPT, 3, 64, 64),
    generator=generator,
    dtype=torch.float32 if DEVICE == "cpu" else torch.float16,
)

In [ ]:
# prompts_A, prompts_B = prepare_prompts_wiki400("data/wiki400.txt", limit=8)
prompts_A, prompts_B = prepare_prompts_wiki398("data/wiki398.txt", limit=8)
# prompts_A, prompts_B = prepare_prompts_mario_bechmark(
#     "/storage2/bartosz/code/t2i2/data/MARIOEval/DrawBenchText/DrawBenchText.txt"
# )


# prompts_A = [{"prompt": "A sign that says 'hate'", "text": "hate"}]
# prompts_B = [{"prompt": "A sign that says 'love'", "text": "love"}]

In [ ]:
len(prompts_A), len(prompts_B)

In [ ]:
ocr_model = get_ocr_easyocr(use_cuda=(DEVICE == "cuda"))

In [ ]:
clip_model, transform = clip.load("ViT-B/32", device=DEVICE, jit=False)
clip_model.eval()

In [ ]:
def sample(
    prompts,
    noise,
    batch_size,
    num_inference_steps,
    generator,
    device,
    run_with_cache,
    attn_idx_to_patch=None,
    attn_heads_idx_to_patch=None,
):
    all_images = np.zeros((len(prompts), 256, 256, 3), dtype=np.float32)
    with tqdm(total=len(prompts)) as pbar:
        for batch_num, batch_start in enumerate(range(0, len(prompts), batch_size)):
            prompt = prompts[batch_start : batch_start + batch_size]
            latent = noise.repeat(len(prompt), 1, 1, 1).to(device)
            pipe.to(device)
            images = pipe(
                prompt=prompt,
                num_inference_steps=num_inference_steps,
                generator=generator,
                latents=latent,
                run_with_cache=run_with_cache,
                attn_idx_to_patch=attn_idx_to_patch,
                batch_num=batch_num,
                attn_heads_idx_to_patch=attn_heads_idx_to_patch,
            ).images
            pipe.to("cpu", silence_dtype_warnings=True)
            pipe_upsample.to(device)
            images = pipe_upsample(
                image=images,
                prompt=prompt,
                generator=generator,
                output_type="np",
            ).images
            pipe_upsample.to("cpu", silence_dtype_warnings=True)
            images = images * 255
            all_images[batch_start : batch_start + batch_size] = images
            pbar.update(len(prompt))
    return all_images


In [ ]:
def calculate_metrics(
    original_images_A,
    original_images_A_feats,
    images,
    texts_A,
    texts_B,
    prompts_A,
    prompts_B,
    device,
    batch_size,
):
    # calculate metrics per sample
    # 1. MSE
    mse = calculate_mse(original_images_A, images)
    # 2.PSNR
    psnr = calculate_psnr_from_mse(mse)
    # 3. SSIM
    ssim_val = ssim(
        torch.from_numpy(original_images_A).permute((0, 3, 1, 2)),
        torch.from_numpy(images).permute((0, 3, 1, 2)),
        data_range=255,
        size_average=False,
    ).numpy()
    # 4. OCR Acc/Prec/Rec
    ocr_texts = [
        get_text_easyocr(ocr_model, images[i].astype(np.uint8)).lower()
        for i in range(images.shape[0])
    ]
    ocr_pr_A, ocr_rec_A, ocr_acc_A = ocr_metrics(ocr_texts, texts_A)
    ocr_pr_B, ocr_rec_B, ocr_acc_B = ocr_metrics(ocr_texts, texts_B)
    # 5. CLIPScore
    image_sim, prompt_A_sim, prompt_B_sim = clip_metrics(
        clip_model,
        images,
        original_images_A_feats,
        device,
        batch_size,
        prompts_A,
        prompts_B,
    )
    # 6. Levenshtein distance
    leve_A = get_levenshtein_distances(ocr_texts, texts_A)
    leve_B = get_levenshtein_distances(ocr_texts, texts_B)
    return {
        "MSE": mse,
        "PSNR": psnr,
        "SSIM": ssim_val,
        "OCR_A_Prec": ocr_pr_A,
        "OCR_A_Rec": ocr_rec_A,
        "OCR_A_Acc": ocr_acc_A,
        "OCR_B_Prec": ocr_pr_B,
        "OCR_B_Rec": ocr_rec_B,
        "OCR_B_Acc": ocr_acc_B,
        "CLIPScore_image": image_sim,
        "CLIPScore_prompt_A": prompt_A_sim,
        "CLIPScore_prompt_B": prompt_B_sim,
        "Levenshtein_A": leve_A,
        "Levenshtein_B": leve_B,
        "Prompts_A": prompts_A,
        "Prompts_B": prompts_B,
        "OCR_texts": ocr_texts,
        "Texts_A": texts_A,
        "Texts_B": texts_B,
    }

Only prompt A

In [ ]:
original_images_A = sample(
    [p["prompt"] for p in prompts_A],
    noises,
    BATCH_SIZE,
    NUM_INFERENCE_STEPS,
    generator,
    DEVICE,
    run_with_cache=False,
)

In [ ]:
original_images_A.shape

In [ ]:
original_images_A_feats = extract_all_images(
    original_images_A, clip_model, DEVICE, batch_size=BATCH_SIZE
)

In [ ]:
original_images_B = sample(
    [p["prompt"] for p in prompts_B],
    noises,
    BATCH_SIZE,
    NUM_INFERENCE_STEPS,
    generator,
    DEVICE,
    run_with_cache=True,
    attn_idx_to_patch=ATTENTIONS_TO_PATCH,
)

In [ ]:
original_images_A_metrics = calculate_metrics(
    original_images_A,
    original_images_A_feats,
    original_images_A,
    [p["text"] for p in prompts_A],
    [p["text"] for p in prompts_B],
    [p["prompt"] for p in prompts_A],
    [p["prompt"] for p in prompts_B],
    DEVICE,
    BATCH_SIZE,
)

original_images_B_metrics = calculate_metrics(
    original_images_A,
    original_images_A_feats,
    original_images_B,
    [p["text"] for p in prompts_A],
    [p["text"] for p in prompts_B],
    [p["prompt"] for p in prompts_A],
    [p["prompt"] for p in prompts_B],
    DEVICE,
    BATCH_SIZE,
)

In [ ]:
original_images_A_df = pd.DataFrame(
    original_images_A_metrics,
)
original_images_A_df["Block_patched"] = ["-" for _ in range(len(prompts_A))]
original_images_B_df = pd.DataFrame(
    original_images_B_metrics,
    index=["Model" for _ in range(len(prompts_A))],
)
original_images_B_df["Block_patched"] = ["Model" for _ in range(len(prompts_B))]

In [ ]:
all_metrics_df = pd.concat([original_images_A_df, original_images_B_df])

In [ ]:
for transformer_idx in tqdm(ATTENTIONS_TO_PATCH):
    patched_images = sample(
        [p["prompt"] for p in prompts_A],
        noises,
        BATCH_SIZE,
        NUM_INFERENCE_STEPS,
        generator,
        DEVICE,
        run_with_cache=False,
        attn_idx_to_patch=transformer_idx,
    )
    patched_images_metrics = calculate_metrics(
        original_images_A,
        original_images_A_feats,
        patched_images,
        [p["text"] for p in prompts_A],
        [p["text"] for p in prompts_B],
        [p["prompt"] for p in prompts_A],
        [p["prompt"] for p in prompts_B],
        DEVICE,
        BATCH_SIZE,
    )

    patched_images_df = pd.DataFrame(
        patched_images_metrics,
    )
    patched_images_df["Block_patched"] = [
        f"T{transformer_idx}" for _ in range(len(prompts_A))
    ]

    all_metrics_df = pd.concat([all_metrics_df, patched_images_df])

In [ ]:
all_metrics_df["Block_patched"].unique()

In [ ]:
all_metrics_df.to_csv("results_if/block_patching.csv", index=False)

In [ ]:
all_metrics_df["Block_patched"].unique()

In [ ]:
all_metrics_df.columns

In [ ]:
all_metrics_df.describe()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(40, 8))

x = np.arange(len(all_metrics_df["Block_patched"].unique()))
width = 0.1  # Width of each bar
cols_to_vis = [
    "OCR_A_Prec",
    "OCR_A_Rec",
    "OCR_A_Acc",
    "OCR_B_Prec",
    "OCR_B_Rec",
    "OCR_B_Acc",
]

for i, col in enumerate(cols_to_vis):
    plt.bar(
        x - (1.5 - i) * width,
        [
            all_metrics_df.loc[all_metrics_df["Block_patched"] == c][col].mean()
            for c in all_metrics_df["Block_patched"].unique()
        ],
        width,
        label=col,
    )

plt.axvline(x=1.5, color="black", linestyle="--")
plt.xticks(x, all_metrics_df["Block_patched"].unique())

plt.title("Transformer block patching")
plt.xlabel("SD3 Transformer Block Patched")
plt.grid(axis="y")
plt.legend()
plt.show()